# FIT 5148: Assignment 1
## Task A. MongoDB Data Model

## The Data Model 
>- The following representation is the sample data model designed for storing data in two separate collections, as follows,

### 1. `hotspot_historic`
`collection_name: hotpost_historic`

**meta-data**: collection used to store data coming from file hotspot_historic.csv



`<model_design>: `







            `{
            '_id': 123
            location: {
                      type: "Point",
                      coordinates: [<long>, <lat>]
                       },
            'datetime':<dummy_value3>, 
            'confidence':<dummy_value4>, 
            'date':<dummy_value5>,
            'surface_temperature_celcius':<dummy_value6>
            },
            
            {
            '_id': 345
            location: {
                      type: "Point",
                      coordinates: [<long>, <lat>]
                       },
            'datetime':<dummy_value3>, 
            'confidence':<dummy_value4>, 
            'date':<dummy_value5>,
            'surface_temperature_celcius':<dummy_value6>
            }
            ...
            
            ...
            {
            '_id': 999
            location: {
                      type: "Point",
                      coordinates: [<long>, <lat>]
                       },
            'datetime':<dummy_value3>, 
            'confidence':<dummy_value4>, 
            'date':<dummy_value5>,
            'surface_temperature_celcius':<dummy_value6>
            }`

### 1.1 sample `hotspot_historic` data
`
{'_id': ObjectId('5cc6820c934369640f86bf8f'),
 'confidence': 78,
 'date': datetime.datetime(2017, 12, 27, 0, 0),
 'datetime': datetime.datetime(2017, 12, 27, 4, 16, 51),
 'location': {'coordinates': [-37.966, 145.051], 'type': 'Point'},
 'surface_temperature': 68}
`
            

### 2. `climate_hotspot`
`collection_name: climate_hotspot`

**meta-data**: collection used to store data coming from file climate_historic.csv



`<model_design>: `



            `{
            'date':{
                    'station'::<dummy_value1>,
                    'date':<dummy_value1>,
                    'air_temperature_celcius':<dummy_value1>,
                    'relative_humidity':<dummy_value1>,
                    'windspeed_knots':<dummy_value1>,
                    'max_wind_speed':<dummy_value1>,
                    'precipitation ':<dummy_value1>,
                    'hotspot_data': ['<list of ids in hotspot_historic collection>']
                    }
            
            }`
            


### 2.1 sample `climate_hotspot` data

`
{'_id': ObjectId('5cc68217934369640f86ca3b'),`

` 'air_temperature': 20,`
 
` 'date': datetime.datetime(2017, 3, 6, 0, 0),`
 
 `'hotspot_data': [{'_id': ObjectId('5cc6820d934369640f86c9f9')},
                  {'_id': ObjectId('5cc6820d934369640f86c9fa')}], `
                  
                 
` 'max_wind_speed': 21.0,`

` 'precipitation': ' 0.00I',`
 
` 'relative_humidity': 52.9,`
 
` 'station': 948701,`
 
` 'windspeed_knots': 17.0}`

## Justification

### Model selection
- The data model for Hotspot historic and Climate historic is based on the **reference** data model.
- Since there can be multiple data-sets of hotspot historic is found for one date value in climate data, **hotspot** data is referenced in **climate** data collection.
- To avoid lengthy document size in single collection, reference based model approach is used.

### Hotspot historic model
- In the Hotspot historic data the **Latitude and Longitude** were combined into lists and refered as points as standard mongoDB spatial data notation.
- Rest of the variables in Hotspot Historic data are straight forward as there is no need to embed the data model.

### Climate hotspot model
- In the Climate Historic the variables are embeded into the **date** key such that for each unique date there are variables from climate historic data refering to the pirticular date and the **IDs** of the Hotspot Historic data falling on the **same** date for the sake of referencing.

### Ease of query
- For the further queries in **Task B** this model was found most appropriate as there was no need to **unwind** the data while running the queries.
- Data is stored in these model in such a way that query retrieval can be done using quite simple queries, avoiding nested and complicated queries.

# Task B. Querying MongoDB using PyMongo
## Task B.1
### Reading data

In [1]:
# Importing the required libraries
import csv  # to read the csv files
from pymongo import MongoClient, ASCENDING, DESCENDING  #The pymongo libraries for accessing the database
import json  # to dump files into the json format
import datetime   # to convert string object to datetime object

In [2]:
#function to read the csv file and return the header and data
def read_csv(file_name):
    header = []
    data = []
    with open(file_name, 'r') as f:
        reader = csv.reader(f)
        row_count = 0
        
        for row in reader:
            if(row_count == 0):
                header.extend(row)
            else:
                data.append(row)
            row_count += 1
    
    return header, data
header_list_climate, data_climate = read_csv('climate_historic.csv')# read the Climate data
header_list_hotspot, data_hotspot = read_csv('hotspot_historic.csv')# read the Hotspot data
print("Header fields for hotspot_historic file are \n{}\n".format(header_list_hotspot))
print("Header fields for climate_historic file are \n{}".format(header_list_climate))

Header fields for hotspot_historic file are 
['latitude', 'longitude', 'datetime', 'confidence', 'date', 'surface_temperature_celcius']

Header fields for climate_historic file are 
['station', 'date', 'air_temperature_celcius', 'relative_humidity', 'windspeed_knots', 'max_wind_speed', 'precipitation ']


### Connection to database

In [3]:
connection = MongoClient()  # Creating a mongoDb connection with localhost and default Port
db = connection['fit5148_assignment_db']  # Creating a database 

## Creation & indexing `hotspot_historic` collection

In [4]:
hotspot_coll = db['hotspot_historic']  #Create a collection Of hotspot historic data

### Creating Indexes for optimizing the data processing 

In [5]:
hotspot_coll.create_index('confidence',unique = False)
hotspot_coll.create_index('surface_temperature', unique = False)

'surface_temperature_1'

In [6]:
# collection index info
hotspot_coll.index_information()

{'_id_': {'key': [('_id', 1)],
  'ns': 'fit5148_assignment_db.hotspot_historic',
  'v': 2},
 'confidence_1': {'key': [('confidence', 1)],
  'ns': 'fit5148_assignment_db.hotspot_historic',
  'v': 2},
 'surface_temperature_1': {'key': [('surface_temperature', 1)],
  'ns': 'fit5148_assignment_db.hotspot_historic',
  'v': 2}}

## Populating  `hotspot_historic` collection

In [7]:
hotspot_list = []  #created a empty list
for d in data_hotspot:  # accessing each value in hotspot data 

    hotspot_dict = {}  #create a empty dictionary everytime the loop starts 
    
    ###Append the data into the dictionary column wise
    hotspot_dict['location'] = {'type':"Point", 'coordinates':[float(d[0]),float(d[1])]}
    hotspot_dict['datetime'] = datetime.datetime.strptime(d[2], '%Y-%m-%dT%H:%M:%S')
    hotspot_dict['confidence'] = int(d[3])
    hotspot_dict['date'] = datetime.datetime.strptime(d[4], '%d/%m/%Y')
    hotspot_dict['surface_temperature'] = int(d[5])
    
    ## Finally append the dictionary into the list created above this will create a list of dictionaries (JSON)
    hotspot_list.append(hotspot_dict)

### Bulk insert

In [8]:
#bulk insert documents into hotspot collection
hotspot_coll.insert_many(hotspot_list, ordered = False)  # Inserting all the values in Hotspot dictionary into the dataframe

### Utility function
* Below functions are utility functions, used to transform data in readable format for query operations.

In [9]:
# Function to convert string date object into the datetime object
def format_date(date):
    return datetime.datetime.strptime(date, '%d/%m/%Y')

In [10]:
def to_dict(cursor):   #A function to return a dictionary from the MongoDb cursor 
    _list = [] #empty list
    for c in cursor:  # for each value in cursor
        _list.append(c)  # append the value into list
    if _list:    # if list is not null
        return _list if len(_list)>1 else _list[0] #return the list if length is >1 else return 0
    else:
        return {}  # return an empty dictionary

In [11]:
# returns hotspot datarows corresponding to a particular date value parameter
def get_hotspotdata_bydate(date):
    _list = []  #emty list
    cursor = hotspot_coll.find({'date':datetime.datetime.strptime(date, '%d/%m/%Y')},{'_id':1})  # query for finding the required date in the hotspot data
    for c in cursor:     # for each value in cursor returned by MongoDb query
        _list.append(c)   #append the value to the list
    return _list              #return the list

## Creation `climate_hotspot` collection

In [12]:
climate_coll = db['climate_hotspot']   #Create a climate historic collection

## Populating `climate_hotspot` collection

In [13]:
climate_list = []  #empty list
for d in data_climate:    #for each in climate data
    climate_dict = {}           #Empty dictionary for every iteration
    
    ###Append the data into the dictionary column wise
    climate_dict['station'] = int(d[0])
    climate_dict['date'] = datetime.datetime.strptime(d[1], '%d/%m/%Y')
    climate_dict['air_temperature'] = int(d[2])
    climate_dict['relative_humidity'] = float(d[3])
    climate_dict['windspeed_knots'] = float(d[4])
    climate_dict['max_wind_speed'] = float(d[5])
    climate_dict['precipitation'] = d[6]
    climate_dict['hotspot_data'] = get_hotspotdata_bydate(d[1])
    
    # finally append into the list to make a list of dictionaries to insert into the DB collection
    climate_list.append(climate_dict)

In [14]:
#bulk insert
climate_coll.insert_many(climate_list, ordered = False)

## Task B.2
### Queries

>- Find climate data on 10th December 2017

In [15]:
# Find climate data on 10th December 2017.
climate_dt = climate_coll.find({'date':format_date('10/12/2017')},{'hotspot_data':0, '_id':0}) #searching for a exact match in date

In [16]:
for m in climate_dt:
    print(m)

{'station': 948702, 'date': datetime.datetime(2017, 12, 10, 0, 0), 'max_wind_speed': 14.0, 'precipitation': ' 0.00I', 'windspeed_knots': 7.3, 'relative_humidity': 53.5, 'air_temperature': 17}


>- Find the latitude, longitude, surface temperature (°C), and confidence when
the surface temperature (°C) was between 65 °C and 100 °C.

In [17]:
#Find the latitude, longitude, surface temperature(°C), and confidence when the 
#surface temperature(°C) was between 65 °C and 100°C
sur_tmp_range = hotspot_coll.find({'surface_temperature':{'$gte':65, '$lte':100}},{'_id':0, 'date':0, 'datetime':0})  #finding data with surface temp between 65 and 100

In [18]:
for m in sur_tmp_range:
    print(m)

{'confidence': 100, 'location': {'coordinates': [-37.642, 149.263], 'type': 'Point'}, 'surface_temperature': 65}
{'confidence': 87, 'location': {'coordinates': [-37.862, 144.175], 'type': 'Point'}, 'surface_temperature': 65}
{'confidence': 90, 'location': {'coordinates': [-37.331, 143.122], 'type': 'Point'}, 'surface_temperature': 65}
{'confidence': 88, 'location': {'coordinates': [-37.46, 148.102], 'type': 'Point'}, 'surface_temperature': 65}
{'confidence': 100, 'location': {'coordinates': [-37.446, 148.102], 'type': 'Point'}, 'surface_temperature': 65}
{'confidence': 100, 'location': {'coordinates': [-37.396, 148.086], 'type': 'Point'}, 'surface_temperature': 65}
{'confidence': 89, 'location': {'coordinates': [-38.3998, 147.064], 'type': 'Point'}, 'surface_temperature': 65}
{'confidence': 89, 'location': {'coordinates': [-38.4792, 146.3081], 'type': 'Point'}, 'surface_temperature': 65}
{'confidence': 89, 'location': {'coordinates': [-36.5661, 142.2956], 'type': 'Point'}, 'surface_tem

>- Find date, surface temperature (°C), air temperature (°C), relative humidity and
max wind speed on 15th and 16th of December 2017.

In [19]:
#getting the data using the IDs and returns all parameters specified in `p_dict` parameter
def hs_obj(_id, params):
    p_dict = {p:1 for p in params}
    return to_dict(hotspot_coll.find({'_id':_id}, p_dict))

In [20]:
# returns the hotspot data corresponding to the ids present in climate data collection.
def get_hs_data(cursor_dict):
    obj_l = []
    obj = {}
    for c in cursor_dict:
        obj = dict(c)
        if 'hotspot_data' in c.keys():
            del obj['hotspot_data']
            for h in c['hotspot_data']:
                obj['surface_temperature'] = hs_obj(h['_id'],['surface_temperature']).get('surface_temperature')
                obj_l.append(obj)
        else:
            obj_l.append(obj)
    return obj_l      

In [21]:
#searching for data of two dates using or operator
cursor_dict = to_dict(climate_coll.find({'$or':[{'date':format_date('15/12/2017')},{'date':format_date('16/12/2017')}]},{'_id':0,'precipitation':0,'station':0,'windspeed_knots':0})) 
date_specific_data = get_hs_data(cursor_dict)

In [22]:
for m in date_specific_data:
    print(m)

{'surface_temperature': 40, 'date': datetime.datetime(2017, 12, 15, 0, 0), 'max_wind_speed': 14.0, 'air_temperature': 18, 'relative_humidity': 52.0}
{'surface_temperature': 40, 'date': datetime.datetime(2017, 12, 15, 0, 0), 'max_wind_speed': 14.0, 'air_temperature': 18, 'relative_humidity': 52.0}
{'surface_temperature': 40, 'date': datetime.datetime(2017, 12, 15, 0, 0), 'max_wind_speed': 14.0, 'air_temperature': 18, 'relative_humidity': 52.0}
{'surface_temperature': 40, 'date': datetime.datetime(2017, 12, 15, 0, 0), 'max_wind_speed': 14.0, 'air_temperature': 18, 'relative_humidity': 52.0}
{'surface_temperature': 57, 'date': datetime.datetime(2017, 12, 16, 0, 0), 'max_wind_speed': 13.0, 'air_temperature': 18, 'relative_humidity': 53.7}
{'surface_temperature': 57, 'date': datetime.datetime(2017, 12, 16, 0, 0), 'max_wind_speed': 13.0, 'air_temperature': 18, 'relative_humidity': 53.7}
{'surface_temperature': 57, 'date': datetime.datetime(2017, 12, 16, 0, 0), 'max_wind_speed': 13.0, 'air_te

>- Find datetime, air temperature (°C), surface temperature (°C) and confidence
when the confidence is between 80 and 100.

In [23]:
conf_range = hotspot_coll.aggregate([
    {
        '$match':{'confidence':{'$gte':80, '$lte':100}}     #confidence between 80 and 100 (inclusive)
    },
    {
        '$lookup':{
            'from':'climate_hotspot',           #Joining on dates the two collection and naming it as climate data
            'localField': 'date',
            'foreignField': 'date',
            'as': 'climate_data'
        }
    },
    {
        '$unwind': '$climate_data'                    #unwinding climate data
    },
    {
        '$project':{                                  #projecting selected values
            '_id':0,
            'datetime':1,
            'confidence':1,
            'surface_temperature':1,
            'air_temperature':1,
            'climate_data.air_temperature': 1
        }
    }
    
])

In [24]:
for doc in conf_range :
    print(doc)

{'confidence': 80, 'climate_data': {'air_temperature': 17}, 'surface_temperature': 54, 'datetime': datetime.datetime(2017, 12, 25, 4, 29, 8)}
{'confidence': 80, 'climate_data': {'air_temperature': 18}, 'surface_temperature': 37, 'datetime': datetime.datetime(2017, 11, 11, 13, 30, 8)}
{'confidence': 80, 'climate_data': {'air_temperature': 13}, 'surface_temperature': 46, 'datetime': datetime.datetime(2017, 10, 20, 4, 41, 19)}
{'confidence': 80, 'climate_data': {'air_temperature': 19}, 'surface_temperature': 46, 'datetime': datetime.datetime(2017, 10, 4, 4, 41, 2)}
{'confidence': 80, 'climate_data': {'air_temperature': 19}, 'surface_temperature': 46, 'datetime': datetime.datetime(2017, 10, 4, 4, 41)}
{'confidence': 80, 'climate_data': {'air_temperature': 17}, 'surface_temperature': 50, 'datetime': datetime.datetime(2017, 9, 23, 4, 59, 11)}
{'confidence': 80, 'climate_data': {'air_temperature': 11}, 'surface_temperature': 54, 'datetime': datetime.datetime(2017, 9, 10, 4, 0)}
{'confidence':

{'confidence': 81, 'climate_data': {'air_temperature': 15}, 'surface_temperature': 54, 'datetime': datetime.datetime(2017, 4, 18, 4, 45, 20)}
{'confidence': 81, 'climate_data': {'air_temperature': 15}, 'surface_temperature': 54, 'datetime': datetime.datetime(2017, 4, 18, 4, 45)}
{'confidence': 81, 'climate_data': {'air_temperature': 15}, 'surface_temperature': 54, 'datetime': datetime.datetime(2017, 4, 18, 4, 44, 50)}
{'confidence': 81, 'climate_data': {'air_temperature': 15}, 'surface_temperature': 60, 'datetime': datetime.datetime(2017, 4, 18, 4, 44, 50)}
{'confidence': 81, 'climate_data': {'air_temperature': 15}, 'surface_temperature': 54, 'datetime': datetime.datetime(2017, 4, 18, 4, 44, 50)}
{'confidence': 81, 'climate_data': {'air_temperature': 15}, 'surface_temperature': 54, 'datetime': datetime.datetime(2017, 4, 18, 4, 44, 50)}
{'confidence': 81, 'climate_data': {'air_temperature': 15}, 'surface_temperature': 54, 'datetime': datetime.datetime(2017, 4, 18, 4, 44, 50)}
{'confiden

>- Find the top 10 records with the highest surface temperature (°C)

In [25]:
#sort data in descending order and project top 10 data
top_surf_tmp = hotspot_coll.find({},{'_id':0}).sort('surface_temperature',DESCENDING).limit(10)

In [26]:
for m in top_surf_tmp:
    print(m)

{'datetime': datetime.datetime(2017, 4, 18, 4, 52), 'confidence': 100, 'location': {'coordinates': [-38.1665, 143.062], 'type': 'Point'}, 'surface_temperature': 124, 'date': datetime.datetime(2017, 4, 18, 0, 0)}
{'datetime': datetime.datetime(2017, 4, 4, 4, 32, 50), 'confidence': 100, 'location': {'coordinates': [-36.343, 142.1986], 'type': 'Point'}, 'surface_temperature': 123, 'date': datetime.datetime(2017, 4, 4, 0, 0)}
{'datetime': datetime.datetime(2017, 5, 1, 4, 14, 20), 'confidence': 100, 'location': {'coordinates': [-36.9318, 143.0907], 'type': 'Point'}, 'surface_temperature': 122, 'date': datetime.datetime(2017, 5, 1, 0, 0)}
{'datetime': datetime.datetime(2017, 3, 18, 3, 50, 50), 'confidence': 100, 'location': {'coordinates': [-37.017, 148.1297], 'type': 'Point'}, 'surface_temperature': 121, 'date': datetime.datetime(2017, 3, 18, 0, 0)}
{'datetime': datetime.datetime(2017, 4, 4, 4, 32, 40), 'confidence': 100, 'location': {'coordinates': [-36.3521, 142.2008], 'type': 'Point'}, '

>- Find the number of fire in each day.

In [27]:
#Find the number of fire in each day. 
num_fire = hotspot_coll.aggregate([{
    '$group': {                                       #grouping the data on date and sum 
        '_id':'$date',
        'number of fires':{'$sum':1}
    }
    
},
{ 
        '$project':{                                  #projecting the selected values
            '_id':0,
            'date': '$_id',
            'number of fires': 1
        }
    }
])

In [28]:
for m in num_fire:
    print(m)

{'number of fires': 2, 'date': datetime.datetime(2017, 3, 6, 0, 0)}
{'number of fires': 8, 'date': datetime.datetime(2017, 3, 10, 0, 0)}
{'number of fires': 5, 'date': datetime.datetime(2017, 3, 12, 0, 0)}
{'number of fires': 2, 'date': datetime.datetime(2017, 3, 13, 0, 0)}
{'number of fires': 7, 'date': datetime.datetime(2017, 3, 15, 0, 0)}
{'number of fires': 6, 'date': datetime.datetime(2017, 3, 17, 0, 0)}
{'number of fires': 21, 'date': datetime.datetime(2017, 3, 19, 0, 0)}
{'number of fires': 2, 'date': datetime.datetime(2017, 3, 24, 0, 0)}
{'number of fires': 54, 'date': datetime.datetime(2017, 3, 28, 0, 0)}
{'number of fires': 22, 'date': datetime.datetime(2017, 3, 31, 0, 0)}
{'number of fires': 7, 'date': datetime.datetime(2017, 4, 1, 0, 0)}
{'number of fires': 118, 'date': datetime.datetime(2017, 4, 6, 0, 0)}
{'number of fires': 72, 'date': datetime.datetime(2017, 4, 3, 0, 0)}
{'number of fires': 39, 'date': datetime.datetime(2017, 4, 7, 0, 0)}
{'number of fires': 357, 'date':

>- Find the average surface temperature(°C) for each day.  

In [29]:
# Find the average surface temperature(°C) for each day.  
avg_st = hotspot_coll.aggregate([{
    '$group': {       #grouping on date and taking averages of all the surface data on pirticular dates
        '_id':'$date',
        'average surface temperature':{'$avg':'$surface_temperature'}          
    }
    
},
{
        '$project':{     #proecting required fields
            '_id':0,
            'date': '$_id',
            'average surface temperature': 1
        }
    }
])

In [30]:
for m in avg_st:
    print(m)

{'average surface temperature': 60.5, 'date': datetime.datetime(2017, 3, 6, 0, 0)}
{'average surface temperature': 69.375, 'date': datetime.datetime(2017, 3, 10, 0, 0)}
{'average surface temperature': 88.2, 'date': datetime.datetime(2017, 3, 12, 0, 0)}
{'average surface temperature': 38.5, 'date': datetime.datetime(2017, 3, 13, 0, 0)}
{'average surface temperature': 46.0, 'date': datetime.datetime(2017, 3, 15, 0, 0)}
{'average surface temperature': 59.5, 'date': datetime.datetime(2017, 3, 17, 0, 0)}
{'average surface temperature': 65.57142857142857, 'date': datetime.datetime(2017, 3, 19, 0, 0)}
{'average surface temperature': 49.0, 'date': datetime.datetime(2017, 3, 24, 0, 0)}
{'average surface temperature': 60.925925925925924, 'date': datetime.datetime(2017, 3, 28, 0, 0)}
{'average surface temperature': 48.72727272727273, 'date': datetime.datetime(2017, 3, 31, 0, 0)}
{'average surface temperature': 46.714285714285715, 'date': datetime.datetime(2017, 4, 1, 0, 0)}
{'average surface temp